In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn import preprocessing
import numpy as np
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics

import tensorflow as tf    
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, LSTM, Input, concatenate, SimpleRNN, Dropout, Bidirectional, Embedding, GRU
from keras.callbacks import History
from keras import regularizers
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
history = History()

In [2]:
df = pd.read_csv("C:\\Data\\Processed_M3\\Sites_10min_time_final.csv")

In [3]:
timeInterval = '10min_time'
# df.columns = ['date', 'site', timeInterval, 'occupancy0', 'speed0', 'volume0','dateTime', 'hour', 'dayOfWeek', 'dayType']
df.columns = ['date','site','timeInterval','occupancy0','speed0','volume0','dateTime','hour','dayOfWeek','dayType','density0','state0']
sites = set(df['site'])
dateTime = pd.to_datetime(df['dateTime'])
df['interval'] = [v.hour*6+int(v.minute/10) for v in dateTime]
df

,date,site,timeInterval,occupancy0,speed0,volume0,dateTime,hour,dayOfWeek,dayType,density0,state0,interval
0,2016-07-01,14005 WB P0,00:00:00,0.000000,100.000000,67.728571,2016-07-01 00:00:00,0,4,0,0.000000,0.0,0
1,2016-07-01,14005 WB P0,00:10:00,0.000000,100.000000,62.821429,2016-07-01 00:10:00,0,4,0,0.000000,0.0,1
2,2016-07-01,14005 WB P0,00:20:00,0.000000,100.000000,53.171429,2016-07-01 00:20:00,0,4,0,0.000000,0.0,2
3,2016-07-01,14005 WB P0,00:30:00,0.000000,100.000000,51.971429,2016-07-01 00:30:00,0,4,0,0.000000,0.0,3
4,2016-07-01,14005 WB P0,00:40:00,0.000000,100.000000,44.350000,2016-07-01 00:40:00,0,4,0,0.000000,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51403,2016-08-21,14063 WB P0,23:10:00,0.890000,97.012222,62.000000,2016-08-21 23:10:00,23,6,1,1.260833,0.0,139
51404,2016-08-21,14063 WB P0,23:20:00,0.436667,96.243056,78.750000,2016-08-21 23:20:00,23,6,1,0.618611,0.0,140
51405,2016-08-21,14063 WB P0,23:30:00,0.490000,96.452778,36.000000,2016-08-21 23:30:00,23,6,1,0.694167,0.0,141
51406,2016-08-21,14063 WB P0,23:40:00,0.520000,96.370833,70.687500,2016-08-21 23:40:00,23,6,1,0.736667,0.0,142


In [4]:
# The lag is the time distance between time-factors and time-target
# If the lag is 0, we say we try to use the data 10 mins ago to forecast the now traffic condition
# If the lag is 2, we say we try to use the data 30 mins ago to forecast the now traffic condition
# If the lag is 5, we say we try to use the data 60 mins ago to forecast the now traffic condition

window = 6*10
lag = 11            # lag 11 means the label is 120 mins later
lag = lag + window

# lag_names = ['volume'+str(t) for t in range(lag+1)]
volume_names = 'volume'+str(lag)
hour_names = 'hour'+str(lag)
dayOfWeek_names = 'dayOfWeek'+str(lag)
interval_names = 'interval'+str(lag)
dayType_names = 'dayType'+str(lag)
data_features = [volume_names, hour_names, dayOfWeek_names, interval_names, dayType_names]

column_names = list(df.columns.values) + data_features
print(column_names)

data = pd.DataFrame()
for site in sites:
    df1 = df[df['site']==site]
    df1.sort_values('dateTime')
    # df1 = pd.concat([df1]+ [df1['volume0'].shift(t) for t in range(lag+1)], axis=1)
    df1 = pd.concat([df1]+ [df1['volume0'].shift(lag)], axis=1)
    df1 = pd.concat([df1]+ [df1['hour'].shift(lag)], axis=1)
    df1 = pd.concat([df1]+ [df1['dayOfWeek'].shift(lag)], axis=1)
    df1 = pd.concat([df1]+ [df1['interval'].shift(lag)], axis=1)
    df1 = pd.concat([df1]+ [df1['dayType'].shift(lag)], axis=1)
    data = data.append(df1)
data.columns = column_names
#data = data.dropna()

features = ['volume' + str(lag)]
features_name = 'volume' + str(lag)
aux_features = [interval_names, dayType_names,hour_names, dayOfWeek_names]
columns = features + aux_features
data

['date', 'site', 'timeInterval', 'occupancy0', 'speed0', 'volume0', 'dateTime', 'hour', 'dayOfWeek', 'dayType', 'density0', 'state0', 'interval', 'volume71', 'hour71', 'dayOfWeek71', 'interval71', 'dayType71']


,date,site,timeInterval,occupancy0,speed0,volume0,dateTime,hour,dayOfWeek,dayType,density0,state0,interval,volume71,hour71,dayOfWeek71,interval71,dayType71
432,2016-07-01,14037 WB P0,00:00:00,1.160000,94.034788,134.000000,2016-07-01 00:00:00,0,4,0,1.643333,0.0,0,NaN,NaN,NaN,NaN,NaN
433,2016-07-01,14037 WB P0,00:10:00,1.323333,91.695000,158.000000,2016-07-01 00:10:00,0,4,0,1.874722,0.0,1,NaN,NaN,NaN,NaN,NaN
434,2016-07-01,14037 WB P0,00:20:00,1.168333,94.034167,136.000000,2016-07-01 00:20:00,0,4,0,1.655139,0.0,2,NaN,NaN,NaN,NaN,NaN
435,2016-07-01,14037 WB P0,00:30:00,0.895000,95.485000,114.000000,2016-07-01 00:30:00,0,4,0,1.267917,0.0,3,NaN,NaN,NaN,NaN,NaN
436,2016-07-01,14037 WB P0,00:40:00,1.328333,94.121111,92.571429,2016-07-01 00:40:00,0,4,0,1.881806,0.0,4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51115,2016-08-21,14049 WB P0,23:10:00,1.492500,97.194861,126.000000,2016-08-21 23:10:00,23,6,1,2.114375,0.0,139,915.0,11.0,6.0,68.0,1.0
51116,2016-08-21,14049 WB P0,23:20:00,1.210000,95.399306,109.000000,2016-08-21 23:20:00,23,6,1,1.714167,0.0,140,1011.0,11.0,6.0,69.0,1.0
51117,2016-08-21,14049 WB P0,23:30:00,0.975000,97.567698,92.000000,2016-08-21 23:30:00,23,6,1,1.381250,0.0,141,928.0,11.0,6.0,70.0,1.0
51118,2016-08-21,14049 WB P0,23:40:00,1.185000,94.434008,94.000000,2016-08-21 23:40:00,23,6,1,1.678750,0.0,142,996.0,11.0,6.0,71.0,1.0


## Helper Functions for Normalising Data

In [5]:
def normalise(columns, df): #IMPORTANT: run this
    """
    Normalise variables given the list 'columns'. Used for SVR model. 
    
    Parameters:
    columns (list): list of strings containing feature names that need to be normalised.
    df (pandas dataframe): training data.
    
    """
    min_max_scaler = preprocessing.MinMaxScaler() #normalise to make variables comparable 
    final_df = pd.DataFrame(min_max_scaler.fit_transform(df[columns]), columns = columns)
    
    for col in df.columns: #add back the columns that weren't normalised
        if col not in columns: 
            final_df[col] = df[col].values
    
#     if 'volume' in columns: #also store the unnormalised travel times if necessary 
#         final_df['volume_unnorm'] = df['volume'].values
    
    return final_df, min_max_scaler

In [6]:
def denormalise(predictions, scaler): #IMPORTANT: run this
    """
    Denormalise the predictions back to the previous scale. 
    """
    predictions = predictions.reshape(1,-1)
    pred = scaler.inverse_transform(predictions)
    
    return pred

In [7]:
def create_auxiliary_input(train_x, test_x, features): 
    """
    Function to filter out only the auxiliary features. 
    Auxiliary features are all the other features other than the lagged features.
    
    """
    
    train_x_aux = train_x[features]
    test_x_aux = test_x[features]
    
    return train_x_aux, test_x_aux

In [8]:
def prepare_ann_data(x):
    """
    Reformat data to appropriate form for the recurrent neural
    network.
    
    """
    train_x = []
    for i in range(0, len(x)):
        row = x.iloc[i][['volume1']].values
        train_x.append(row)
    return np.array(train_x)

## Helper Functions for Window the data

In [9]:
def window_generator(x,y,lag,window):
    """
    1. we assume the train test is dataframe format, 
    and it contains both time series and other features
       
    2. this is binary classification thus no normalization apply to label. 
    and we don't need the base line like time series forecasting
    
    3. both ANN and stats mode will be processed in same manner as (1,n_features).
    it is a flatten version for multiple window input and can be accepted by both ANN and stats model.
    """
    local_x = x.copy()
    local_y = y.copy()
    
    index_x = local_x.reset_index().index
    index_y = local_y.reset_index().index
    data_n = len(x) - lag               # for 5 window 0 lag, means 5 time interval before points predict 1 time interval later.

    y_windowed = []
    x_windowed = []
    
    for i in range(data_n):
        x_windowed.append(local_x[i : i+window].values.tolist())  # get features
        y_windowed.append(local_y[index_y[i]])
        if i == 0:
            print()
#             print("*    processing the time series " + str(window) +" window in lag " + str(lag-window))
#             print('features: ', local_x[i : i+window], ' in ', str(index_x[0]),  ' to ', str(index_x[0]+window))
#             print('label: ', local_y[index_y[i]], ' in ', str(index_y[0]))

    y_windowed = np.asarray(y_windowed).astype(np.float32)
    
    # reshape to (n_sample, n_feautre * n_length) still 2d
    x_windowed = np.asarray(x_windowed).astype(np.float32)
#     print('input shape before:', x_windowed.shape)
#     #x_windowed = np.reshape(x_windowed, (x_windowed.shape[0],-1))
#     print('input shape after:', x_windowed.shape)
    
    
    return x_windowed, y_windowed

 ## Experiment

In [10]:
def evaluation(x,y):
#     print("accuracy_score", accuracy_score(x,y))
    print(metrics.classification_report(x,y))
#     print("roc_auc_score: ", metrics.roc_auc_score(x,y))
#     print("f1 score: ", metrics.f1_score(x,y))

In [11]:
%%time

lstm_pre,lstm_test=[],[]



for siteId in sites:
    print('\n',siteId)
    tmp = data[data['site']==siteId]
    tmp = tmp.dropna()
    tmp = tmp.reset_index()
    if len(tmp) == 0:
        #print('error',siteId)
        continue
    
    index = len(tmp) * 0.80
    train = tmp.loc[:index]
    test = tmp.loc[index-1:]
    len(train)/len(tmp) * 100 
    
    ### Normalise all data (simulated and real accidents dataset) and create lags.
    train_norm, _ = normalise(features,train)
    test_norm, _ = normalise(features,test)

    # this is a binary classification thus we don't normlize the label
    y_train = train_norm['state0']
    x_train = train_norm[features]
    y_test = test_norm['state0']
    x_test = test_norm[features]

    x_train_win, y_train_win = window_generator(x_train,y_train,lag,window)
    x_test_win, y_test_win = window_generator(x_test,y_test,lag,window)
    
#     x_train_win = np.reshape(x_train_win, (x_train_win.shape[0], x_train_win.shape[1]))
#     x_test_win = np.reshape(x_test_win, (x_test_win.shape[0], x_test_win.shape[1]))  
    
    # LSTM
    model = Sequential()
    model.add(LSTM(3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.fit(x_train_win, y_train_win, epochs=100, verbose=0)
    y_pred = model.predict(x_test_win)
    y_pred = np.round(y_pred).astype(int)
    lstm_pre.extend(y_pred.tolist())
    lstm_test.extend(y_test_win.tolist())
    



 14037 WB P0



 14011 WB P0



 14005 WB P0



 14063 WB P0



 14025 WB P0



 14055 WB P0



 14049 WB P0


Wall time: 19min 34s


In [12]:
%%time

mrp_pre,mrp_test=[],[]
lr_pre,lr_test=[],[]
rf_pre,rf_test=[],[]
svc_pre,svc_test=[],[]
kn_pre,kn_test=[],[]
xgb_pre,xgb_test=[],[]
dl_pre,dl_test=[],[]
rnn_pre,rnn_test=[],[]
#lstm_pre,lstm_test=[],[]
gru_pre,gru_test=[],[]
bilstm_pre,bilstm_test=[],[]


for siteId in sites:
    print('\n',siteId)
    tmp = data[data['site']==siteId]
    tmp = tmp.dropna()
    tmp = tmp.reset_index()
    if len(tmp) == 0:
        #print('error',siteId)
        continue
    
    index = len(tmp) * 0.80
    train = tmp.loc[:index]
    test = tmp.loc[index-1:]
    len(train)/len(tmp) * 100 
    
    ### Normalise all data (simulated and real accidents dataset) and create lags.
    train_norm, _ = normalise(features,train)
    test_norm, _ = normalise(features,test)

    # this is a binary classification thus we don't normlize the label
    y_train = train_norm['state0']
    x_train = train_norm[features]
    y_test = test_norm['state0']
    x_test = test_norm[features]

    x_train_win, y_train_win = window_generator(x_train,y_train,lag,window)
    x_test_win, y_test_win = window_generator(x_test,y_test,lag,window)

    
    print('RNN')
    # RNN
    model = Sequential()
    model.add(SimpleRNN(3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.fit(x_train_win, y_train_win, epochs=100, verbose=0)
    y_pred = model.predict(x_test_win)
    y_pred = np.round(y_pred).astype(int)
    rnn_pre.extend(y_pred.tolist())
    rnn_test.extend(y_test_win.tolist())
    
    print('LSTM')
    # LSTM
    model = Sequential()
    model.add(LSTM(3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.fit(x_train_win, y_train_win, epochs=100, verbose=0)
    y_pred = model.predict(x_test_win)
    y_pred = np.round(y_pred).astype(int)
    lstm_pre.extend(y_pred.tolist())
    lstm_test.extend(y_test_win.tolist())
    
    print('BiLSTM')
    # BiLSTM
    model = Sequential()
    model.add(Bidirectional(LSTM(3)))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.fit(x_train_win, y_train_win, epochs=100, verbose=0)
    y_pred = model.predict(x_test_win)
    y_pred = np.round(y_pred).astype(int)
    bilstm_pre.extend(y_pred.tolist())
    bilstm_test.extend(y_test_win.tolist())
    
    print('GRU')
    # GRU
    model = Sequential()
    model.add(GRU(3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    model.fit(x_train_win, y_train_win, epochs=100, verbose=0)
    y_pred = model.predict(x_test_win)
    y_pred = np.round(y_pred).astype(int)
    gru_pre.extend(y_pred.tolist())
    gru_test.extend(y_test_win.tolist())


 14037 WB P0


RNN
LSTM
BiLSTM
GRU

 14011 WB P0


RNN
LSTM
BiLSTM
GRU

 14005 WB P0


RNN
LSTM
BiLSTM
GRU

 14063 WB P0


RNN
LSTM
BiLSTM
GRU

 14025 WB P0


RNN
LSTM
BiLSTM
GRU

 14055 WB P0


RNN
LSTM
BiLSTM
GRU

 14049 WB P0


RNN
LSTM
BiLSTM
GRU
Wall time: 2h 29min 53s


In [13]:
# methods = [('MRP',mrp_pre,mrp_test),('DL',dl_pre,dl_test),('LR',lr_pre,lr_test),('Random Forest',rf_pre,rf_test),('SVC',svc_pre,svc_test),('Kneighbours',kn_pre,kn_test),('xgb',xgb_pre,xgb_test)]
# methods = [('LR',lr_pre,lr_test),('DL',dl_pre,dl_test),('SVC',svc_pre,svc_test),('xgb',xgb_pre,xgb_test),('MRP',mrp_pre,mrp_test)]
# methods = [('xgb',xgb_pre,xgb_test)]
methods = [('RNN',rnn_pre,rnn_test),('LSTM',lstm_pre,lstm_test),('BiLSTM',bilstm_pre,bilstm_test),('GRU',gru_pre,gru_test)]

for method in methods:
    print(method[0])
    evaluation(method[1],method[2])
    

RNN
              precision    recall  f1-score   support

           0       0.98      0.95      0.96      9034
           1       0.50      0.71      0.59       654

    accuracy                           0.93      9688
   macro avg       0.74      0.83      0.78      9688
weighted avg       0.95      0.93      0.94      9688

LSTM
              precision    recall  f1-score   support

           0       0.98      0.97      0.98     17782
           1       0.69      0.81      0.75      1594

    accuracy                           0.95     19376
   macro avg       0.84      0.89      0.86     19376
weighted avg       0.96      0.95      0.96     19376

BiLSTM
              precision    recall  f1-score   support

           0       0.98      0.97      0.98      8840
           1       0.72      0.79      0.76       848

    accuracy                           0.96      9688
   macro avg       0.85      0.88      0.87      9688
weighted avg       0.96      0.96      0.96      9688

GRU